In [1]:
#include <mlpack/xeus-cling.hpp>
#include <mlpack/core.hpp>
#include <mlpack/methods/linear_regression/linear_regression.hpp>
#include <mlpack/core/data/split_data.hpp>
#define WITHOUT_NUMPY 1
#include "matplotlibcpp.h"
#include "xwidgets/ximage.hpp"

namespace plt = matplotlibcpp;
using namespace mlpack;


In [2]:
!wget https://matrix.org/_matrix/media/r0/download/matrix.org/WvrgbgzkyIMbvkxLkKKNyMrO/housing.csv

--2021-06-21 12:15:54--  https://matrix.org/_matrix/media/r0/download/matrix.org/WvrgbgzkyIMbvkxLkKKNyMrO/housing.csv
Resolving matrix.org (matrix.org)... 104.20.21.236, 104.20.20.236, 2606:4700:10::6814:14ec, ...
Connecting to matrix.org (matrix.org)|104.20.21.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/csv]
Saving to: ‘housing.csv.1’

     0K .......... .......... .......... .......... ..........  3% 5.65M 0s
    50K .......... .......... .......... .......... ..........  7% 4.87M 0s
   100K .......... .......... .......... .......... .......... 10% 14.6M 0s
   150K .......... .......... .......... .......... .......... 14% 10.2M 0s
   200K .......... .......... .......... .......... .......... 17% 10.0M 0s
   250K .......... .......... .......... .......... .......... 21% 11.0M 0s
   300K .......... .......... .......... .......... .......... 25% 11.7M 0s
   350K .......... .......... .......... .......... .......... 28% 6.42M 0

In [7]:
!sed 1d housing.csv > housing_without_header.csv

In [2]:
arma::mat dataset;
data::DatasetInfo info;
info.Type(9) = mlpack::data::Datatype::categorical;
data::Load("housing_without_header.csv", dataset, info);

In [3]:
// Print the first 5 rows of the input data.
std::cout << dataset.submat(0, 0, dataset.n_rows - 1 , 5)<< std::endl;

  -1.2223e+02  -1.2222e+02  -1.2224e+02  -1.2225e+02  -1.2225e+02  -1.2225e+02
   3.7880e+01   3.7860e+01   3.7850e+01   3.7850e+01   3.7850e+01   3.7850e+01
   4.1000e+01   2.1000e+01   5.2000e+01   5.2000e+01   5.2000e+01   5.2000e+01
   8.8000e+02   7.0990e+03   1.4670e+03   1.2740e+03   1.6270e+03   9.1900e+02
            0   1.0000e+00   2.0000e+00   3.0000e+00   4.0000e+00   5.0000e+00
   3.2200e+02   2.4010e+03   4.9600e+02   5.5800e+02   5.6500e+02   4.1300e+02
   1.2600e+02   1.1380e+03   1.7700e+02   2.1900e+02   2.5900e+02   1.9300e+02
   8.3252e+00   8.3014e+00   7.2574e+00   5.6431e+00   3.8462e+00   4.0368e+00
   4.5260e+05   3.5850e+05   3.5210e+05   3.4130e+05   3.4220e+05   2.6970e+05
            0            0            0            0            0            0



In [4]:
std::cout << dataset(4,20639);

841

In [6]:
#include<mlpack/core/data/one_hot_encoding.hpp>
arma::mat encoded_dataset; 
data::OneHotEncoding(dataset, encoded_dataset, info);

In [7]:
encoded_dataset.n_rows

1937

In [2]:
#include "/home/viole/tacd/examples/utils/pandasscatter.hpp"
PandasScatter("housing.csv", "longitude", "latitude", "output.png");
auto im = xw::image_from_file("output.png").finalize();
im

A Jupyter widget with unique id: bdff7382f4154441b6f6ae8d77f22ebd

In [10]:
encoded_dataset.n_rows

1937

In [11]:
dataset.n_rows

10

In [3]:
PandasScatterColor("housing.csv","longitude","latitude","Population","median_house_value","output1.png");
auto im = xw::image_from_file("output1.png").finalize();
im

A Jupyter widget with unique id: ad52ef49246a4ba186ea82a6fc6186c7

In [4]:
PandasScatterMap("housing.csv","california.png","longitude","latitude","Population","median_house_value","output2.png");
auto im = xw::image_from_file("output2.png").finalize();
im

A Jupyter widget with unique id: d6a0761626a84e90b71cb12c548d39bb

In [100]:
// const arma::Col<size_t> indices = {'9'}

In [6]:
// Labels are the last row.
arma::rowvec labels =
    arma::conv_to<arma::rowvec>::from(encoded_dataset.row(encoded_dataset.n_rows - 2));
encoded_dataset.shed_row(encoded_dataset.n_rows - 2);

In [7]:
arma::mat trainSet, testSet;
arma::rowvec trainLabels, testLabels;


In [8]:

// Split dataset randomly into training set and test set.
data::Split(encoded_dataset, labels, trainSet, testSet, trainLabels, testLabels,
    0.2 /* Percentage of dataset to use for test set. */);

In [9]:
using namespace mlpack::regression;

In [10]:
LinearRegression lr(trainSet, trainLabels, 0.0);

In [11]:
lr.Train(trainSet, trainLabels);

In [12]:
arma::rowvec output; 
lr.Predict(testSet, output);

In [13]:
lr.ComputeError(trainSet, trainLabels);

In [14]:
std::cout<<lr.ComputeError(trainSet, trainLabels);

1.4399e-05

In [15]:
std::cout<<testLabels[1] << std:: endl;
std::cout<<output[1]<<std::endl;

0
-0.00710056


In [16]:
std:: cout << arma::min(testLabels);

0

In [17]:
std::cout<< testLabels.n_cols;

4128

In [18]:
arma::vec test_labels_col = testLabels.t();
arma::vec output_col = output.t();

In [19]:
test_labels_col.print()

        0
        0
   1.0000
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
   1.0000
        0
        0
        0
   1.0000
   1.0000
        0
        0
        0
        0
        0
        0
        0
   1.0000
        0
   1.0000
   1.0000
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
   1.0000
   1.0000
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
        0
   1.0000
        0
        0
        0
        0
   1.0000
        0
        0
        0
        0
        0
        0
        0
        0
        0


In [20]:
output_col.print()

  -7.1807e-04
  -7.1006e-03
   9.9861e-01
   3.6002e-03
   9.4113e-03
   1.2611e-03
  -1.5570e-04
  -2.8432e-03
   8.8812e-03
  -6.7749e-03
  -1.3892e-02
   4.4057e-05
  -4.2409e-03
  -4.4199e-04
   1.3952e-03
  -1.1014e-03
   8.6403e-03
  -1.5439e-03
   9.9491e-01
   1.5102e-03
   3.7455e-03
   6.9404e-03
   9.9385e-01
   9.9737e-01
  -3.9220e-03
   8.8375e-04
   2.3568e-03
   1.6878e-03
  -1.1246e-03
  -5.8154e-04
  -3.1786e-03
   1.0024e+00
   5.9394e-03
   9.9962e-01
   1.0050e+00
  -9.2859e-04
   7.7892e-04
   3.6099e-04
  -3.4161e-03
   6.4888e-04
   5.9918e-03
  -1.7098e-03
  -1.3003e-04
   3.4526e-03
   3.8487e-03
  -2.5831e-03
   3.1083e-03
   1.8576e-03
   6.0922e-03
   3.2470e-03
   3.0494e-03
  -1.3078e-03
  -3.3968e-03
  -6.3978e-03
   5.5327e-03
  -1.0877e-03
   5.6500e-03
   5.3531e-05
   1.5467e-03
  -2.1822e-03
  -2.4890e-04
   8.7353e-04
   4.2288e-03
   8.0666e-04
   2.1985e-03
  -9.0655e-04
   9.3137e-04
   8.4289e-04
  -1.7630e-03
  -2.6706e-03
   1.0057e+00
   1.0